In [1]:
import tpn.util as tu

In [66]:
from tpn.clipboard import cb

In [67]:
size = 1024209543168
default_block_size = 65536

In [68]:
class Dd(tu.SlotObject):
    __slots__ = [
        'bs',
        'skip',
        'seek',
        'count',
        'skipped_bytes',
        'seeked_bytes',
        'total_bytes',
    ]
    def __init__(self, bs, skip, seek, count):
        self.bs = int(bs)
        self.skip = int(skip)
        self.seek = int(seek)
        self.count = int(count)
        self.skipped_bytes = self.skip * self.count
        self.seeked_bytes = self.seek * self.count
        self.total_bytes = self.bs * self.count

In [69]:
def slice_blocks(size, count):
    from math import floor
    block_size = floor(size / count)
    remainder = size - (block_size * count)
    blocks = [ block_size for _ in range(count) ]
    blocks[-1] += remainder
    return blocks
    

In [70]:
from tpn.util import is_power_of_2

def get_dds(total_bytes, num_procs, block_size=65536, min_block_size=4096):
    assert min_block_size < block_size, (min_block_size, block_size)
    assert min_block_size > 0, (min_block_size)
    assert is_power_of_2(min_block_size), (min_block_size)
    assert block_size % min_block_size == 0
        
    remaining_bytes = total_bytes % block_size
    parallel_bytes = total_bytes - remaining_bytes
    
    is_perfect_fit = (remaining_bytes == 0)
    if is_perfect_fit:
        actual_procs = num_procs
        remaining_block_count = 0
        remaining_block_multiplier = 0
    else:
        actual_procs = num_procs + 1
        assert remaining_bytes % min_block_size == 0, (remaining_bytes % min_block_size)
        remaining_block_count = remaining_bytes / min_block_size
        remaining_block_multiplier = block_size / min_block_size

    assert actual_procs >= 1, actual_procs

    parallel_block_count = parallel_bytes / block_size
    slices = slice_blocks(parallel_block_count, num_procs)
    assert len(slices) == num_procs, (len(slices), num_procs)
    #ipdb.set_trace()
    
    dds = list()
    start = 0
    end = 0
    for (i, slice) in enumerate(slices):
        if i == 0:
            offset = 0
        else:
            offset = sum(slices[:i])
        count = slices[i]
        dd = Dd(
            bs=block_size,
            skip=offset,
            seek=offset,
            count=count,
        )
        dds.append(dd)
    
    if not is_perfect_fit:
        offset = sum(slices)
        dd = Dd(
            bs=min_block_size,
            skip=offset,
            seek=offset,
            count=remaining_block_count
        )
        dds.append(dd)
        
    total_check = sum(dd.total_bytes for dd in dds)
    assert total_bytes == total_check, (total_bytes, total_check)
        
    return dds

In [71]:
dds = get_dds(size, 12)

In [72]:
def convert_dd_to_command(dd, input_file, output_file):
    cmd = (
        f'dd if={input_file} of={output_file} '
        f'bs={dd.bs} skip={dd.skip} seek={dd.seek} '
        f'count={dd.count} status=progress &'
    )
    return cmd

In [73]:
text = '\n'.join([ convert_dd_to_command(dd, '/dev/nvd0', '/dev/nvd1') for dd in dds ])
cb(text)
print(text)

copied 1238 characters into clipboard...
dd if=/dev/nvd0 of=/dev/nvd1 bs=65536 skip=0 seek=0 count=1302349 status=progress &
dd if=/dev/nvd0 of=/dev/nvd1 bs=65536 skip=1302349 seek=1302349 count=1302349 status=progress &
dd if=/dev/nvd0 of=/dev/nvd1 bs=65536 skip=2604698 seek=2604698 count=1302349 status=progress &
dd if=/dev/nvd0 of=/dev/nvd1 bs=65536 skip=3907047 seek=3907047 count=1302349 status=progress &
dd if=/dev/nvd0 of=/dev/nvd1 bs=65536 skip=5209396 seek=5209396 count=1302349 status=progress &
dd if=/dev/nvd0 of=/dev/nvd1 bs=65536 skip=6511745 seek=6511745 count=1302349 status=progress &
dd if=/dev/nvd0 of=/dev/nvd1 bs=65536 skip=7814094 seek=7814094 count=1302349 status=progress &
dd if=/dev/nvd0 of=/dev/nvd1 bs=65536 skip=9116443 seek=9116443 count=1302349 status=progress &
dd if=/dev/nvd0 of=/dev/nvd1 bs=65536 skip=10418792 seek=10418792 count=1302349 status=progress &
dd if=/dev/nvd0 of=/dev/nvd1 bs=65536 skip=11721141 seek=11721141 count=1302349 status=progress &
dd if=/